# Pràctica 2

L'objectiu d'aquesta segona pràctica es demostrar que heu assolit els conceptes que s'han explicar a l'assignatura i s'han practicat a les sessions presencials, relacionats amb el disseny i l'ús de xarxes neurals. 

Aquesta pràctica consta de 3 enunciats dels quals només heu de realitzar el que us hagi tocat per sorteig.

**Condicions**
1. El model que solucioni el problema estarà basat en xarxes neurals, aquestes s'han d'entrenar i avaluar emprant la llibreria _Pytorch_.
2. Es demana que com a mínim s'avaluin 2 models diferents: un que ha d'estar creat per vosaltres i un altre que es basi en una xarxa ja existent. Evidentment es permeten modificacions de la ja existen per adaptar-ho al problema que es vol resoldre.
3. El resultat del treball serà un informe on s'expliqui el procés que s'ha dut a terme per arribar a la que considereu que és millor solució. El document serà en format `pdf`. Podreu adjuntar una carpeta amb el codi i recursos que trobeu necessaris per comprovar la veracitat del que explicau al document.
4. Aquest document ha de tenir un llenguatge formal i tècnic i ha d'estar correctament estructurat:
    - Introducció al problema
    - Solucions considerades (dades, característiques, models, mètriques)
    - Experiments realitzats
    - Resultats dels experiments
    - Conclusions
5. A més del document explicatiu s'ha d'adjuntar un fitxer amb els pesos del millor entrenament de cada una de les xarxes que heu emprat (la que heu dissenyat vosaltres i la que ja existia), de tal manera que el professor pugui validar els resultats sense haver de repetir l'entrenament. Sense l'adjunció d'aquests fitxers la pràctica no es podrà aprovar.
6. Les dades depenen de cada un dels tres enunciats i les trobareu en el seu apartat.


**Avaluació**

- El treball es durà a terme en parelles.
- El professor es reserva la possibilitat de convocar als grups a una revisió de la pràctica de forma presencial.
- Només està permés emprar tècniques de disseny i entrenament vistes a classe.
- Tot el que no està fet pels alumnes ha d'estar referenciat, en cas contrari es considerarà com una còpia.


**Data d'entrega**

- Aquest treball s'entrega dia 15 de gener.
- Es realitzarà una tutoria dilluns 9 de gener a les 15:30.
---

## Enunciat 1: Classificació

El problema que heu de resoldre en aquest cas és un problema de classificació amb el conjunt de dades _Horses or Human_ dataset [enllaç](https://laurencemoroney.com/datasets.html). És un conjunt de dades generat per ordinador en el que trobareu dues classes diferents: persones i cavalls (500 imatges de cavalls i 547 imatges de persones). També dos subconjunts de dades ja definits: entrenament  i validació. Les imatges tenen una mida de 300x300 pixels i es troben en RGB.

A més de la feina de classificació i presentació dels resultats amb el conjunt de dades que es proporciona, també es demana que construiu un petit conjunt d'imatges (entre 10 i 20) de persones i cavalls reals com a conjunt de test i obtingueu les mesures rendiment adients per aquestes dades.


**Exemples del dataset**
<div style="display:flex">
     <div style="flex:1;padding-right:10px;">
          <img src="img/human01-16.png" width="200"/>
     </div>
     <div style="flex:1;padding-left:10px;">
          <img src="img/horse03-3.png" width="200"/>
     </div>
</div>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class HumanHorseDataset(Dataset):

    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform

    
       
    #  def load_data(self):
    #     for file in os.listdir(self.path):
    #         if file.endswith(".png"):
    #             image = Image.open(os.path.join(self.path, file))
    #             if self.transform is not None:
    #                 image = self.transform(image)
    #             self.data.append(image)
    #             self.labels.append(0 if file.startswith("horse") else 1)
   
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.path,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, imgs_path):
        self.imgs_path = imgs_path
        file_list = glob.glob(self.imgs_path + "*")
        print(file_list)
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.png"):
                self.data.append([img_path, class_name])
        print(self.data)
        self.class_map = {"horses" : 0, "humans": 1}
        self.img_dim = (300, 300)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor([class_id])
        return img_tensor, class_id

In [ ]:
train_batch_size = 64
test_batch_size = 100

dataset = CustomDataset("data/train/")
data_loader = DataLoader(dataset, batch_size=train_batch_size, shuffle=True)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])

# Creamos los datasets de entrenamiento y test, estan dentro de data/train
# y data/test
